In [2]:
import json
import pandas as pd
import re
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

from tensorflow.keras.models import load_model

import urllib3
import ast


In [ ]:
# json파일 불러옴 (85개 파일-> 35)
len_rawdata=0
raw_data={"utterance":[]}

for k in range(0,3000):
    if k < 10:
        name="DKSR2000000"+str(k)+".json"
    elif k <= 10 and k <100:
        name="DKSR200000"+str(k)+".json"
    elif 100<= k and k<890:
        name="DKSR20000"+str(k)+".json"
    elif 1700<= k and k<7350:
        name="DKSR2000"+str(k)+".json"
    try:
        f = open("[라벨]경상도_학습데이터_1/"+name, encoding="UTF-8")
        data = json.loads(f.read())
        raw_data["utterance"].extend(data["utterance"])
        #print(data["speaker"][1]["age"],data["speaker"][1]["birthplace"],data["speaker"][1]["principal_residence"])
        len_rawdata += 1

    except:
        print(k, "실패")
        pass    

In [169]:
with open('raw_data_3000.pickle','wb') as fw:
    pickle.dump(raw_data, fw)

In [ ]:
#59.9s

In [173]:
with open('raw_data_3000.pickle', 'rb') as fr:
    raw_data = pickle.load(fr)

In [13]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    all_std_word=[]

    filter = re.compile('[^가-힣+]')    
    
    for i in range(len(raw_data["utterance"])):
        #print(raw_data["utterance"])
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                # print(eojeol[k]["eojeol"])
                # print(dialect_word)
                if filter.sub('', eojeol[k]['eojeol']) not in dialect_word :
                    dlt = filter.sub('', eojeol[k]['eojeol'])
                    std = filter.sub('', eojeol[k]['standard'])
                    if dlt=="" or std=="":
                        #print("dd", eojeol[k]['eojeol'], "dd")
                        pass
                    else:
                        dialect_word.append(dlt)
                        standard_word.append(std)
            else:
                if eojeol[k]["standard"] not in standard_word:
                    word = filter.sub('', eojeol[k]['standard'])
                    if word == "":
                        pass
                        #print("dd", eojeol[k]['eojeol'], "dd")
                    else:
                        all_std_word.append(word)

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']

    index_to_dialect = dialect_dictionary.to_dict()
    index_to_standard = standard_dictionary.to_dict() #{인덱스:"단어"}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}#{"단어":인덱스}
    
    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard,all_std_word   

In [14]:
dialect_to_index,standard_to_index,index_to_dialect,index_to_standard,all_std_word = make_dict(raw_data)


In [103]:
#9m9.9s

In [180]:
type(dialect_to_index)
# type(standard_to_index)
# type(index_to_dialect)
# type(index_to_standard)
# type(all_std_word)

dict

In [182]:
with open('dialect_to_index.pickle','wb') as fw:
    pickle.dump(dialect_to_index, fw)

with open('standard_to_index.pickle','wb') as fw:
    pickle.dump(standard_to_index, fw)

with open('index_to_dialect.pickle','wb') as fw:
    pickle.dump(index_to_dialect, fw)

with open('index_to_standard.pickle','wb') as fw:
    pickle.dump(index_to_standard, fw)

In [183]:
with open('dialect_to_index.pickle', 'rb') as fr:
    dialect_to_index = pickle.load(fr)

with open('standard_to_index.pickle', 'rb') as fr:
    standard_to_index = pickle.load(fr)

with open('index_to_dialect.pickle', 'rb') as fr:
    index_to_dialect = pickle.load(fr)

with open('index_to_standard.pickle', 'rb') as fr:
    index_to_standard = pickle.load(fr)

list

In [15]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 
                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect_idx = dialect_to_index[dialect]
                    #standard_idx = standard_to_index[standard]
                    standard_idx = [k for k, v in index_to_standard.items() if v == standard][0]

                    dialect_form.append(dialect_idx)
                    standard_form.append(standard_idx)
                except:
                    pass
            
    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})

    return result               

In [16]:
pairs = make_pair(raw_data)
pairs[:10]

,dialect_words,standard_words
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,1,1
6,5,5
7,6,1
8,1,1
9,7,7


In [184]:
pairs.to_csv("pairs.csv")

In [5]:
pairs = pd.read_csv("pairs.csv")
pairs = pairs[["dialect_words", "standard_words"]]

In [18]:
# 사투리 사전에 존재하는지 확인하는 함수
def exist_in_dialect(x):

    if x in dialect_to_index:
        return True
    else:
        return False

In [19]:
trans_xtrain = np_utils.to_categorical(pairs["dialect_words"])
trans_ytrain = np_utils.to_categorical(pairs["standard_words"])
len(trans_xtrain[0])

7581

In [17]:
len(trans_xtrain[0])

7581

In [20]:
translate_model = Sequential()
translate_model.add(Dense(256, input_dim = 7581, activation="relu"))
translate_model.add(Dense(128, input_dim = 7581, activation="relu"))
translate_model.add(Dense(7581, activation="softmax"))

In [21]:
translate_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
translate_model.fit(trans_xtrain, trans_ytrain, batch_size=100, epochs=50)

Epoch 1/50
1193/1193 [==============================] - 64s 53ms/step - loss: 1.6969 - accuracy: 0.7734
Epoch 2/50
1193/1193 [==============================] - 69s 58ms/step - loss: 0.8077 - accuracy: 0.8867
Epoch 3/50
1193/1193 [==============================] - 69s 58ms/step - loss: 0.6309 - accuracy: 0.9063
Epoch 4/50
1193/1193 [==============================] - 68s 57ms/step - loss: 0.5306 - accuracy: 0.9173
Epoch 5/50
1193/1193 [==============================] - 65s 55ms/step - loss: 0.4568 - accuracy: 0.9246
Epoch 6/50
1193/1193 [==============================] - 73s 61ms/step - loss: 0.3965 - accuracy: 0.9313
Epoch 7/50
1193/1193 [==============================] - 80s 67ms/step - loss: 0.3425 - accuracy: 0.9366
Epoch 8/50
1193/1193 [==============================] - 82s 69ms/step - loss: 0.2922 - accuracy: 0.9409
Epoch 9/50
1193/1193 [==============================] - 81s 68ms/step - loss: 0.2437 - accuracy: 0.9464
Epoch 10/50
1193/1193 [==============================] - 71s 60m

In [206]:
translate_model.save("translate_model_1.h5")

In [ ]:
#58분

In [9]:
# 사투리 단어 번역 함수
def translate(word):
    index = dialect_to_index[word]
    one_hot = np_utils.to_categorical(index,len(trans_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
    #print(translate_model.predict(np.array([one_hot])))
    pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
    print(pred_index)
    #print(len(translate_model.predict(np.array([one_hot]))[0]))
    res = index_to_standard[pred_index]

    return res

In [24]:
translate("인자")

85


'이제'

In [10]:
# 품사 사전
a_tag = ['NP','VP','AP','VNP','DP','IP','X','L','R']
b_tag = ['SBJ','OBJ','MOD','AJT','CMP','CNJ']
c_tag = []
pos_dict = {}
for i in a_tag:
    for j in b_tag:
        c_tag.append(i+"_"+j)
c_tag.extend(a_tag)
c_tag.extend(b_tag)
for i in range(len(c_tag)):
    pos_dict[c_tag[i]] = i

In [13]:
# 데이터의 전체 어절을 딕셔너리로 만듦
def make_eojeol(index_to_standard, dialect_to_index):

    eojeol_list=[]
    st_list = []
    di_list = []
    
    for k in index_to_standard.values():
        st_list.append(k)
    print(len(st_list))
 

    for v in dialect_to_index:
        di_list.append(v)
    print(len(di_list))    

    for i, j in zip(st_list, di_list):
        eojeol_list.append(i)
        eojeol_list.append(j + "0") #사투리에는 0을 붙이기
        
    # for i in all_std_word:
    #     if i not in eojeol_list:
    #         eojeol_list.append(i) #사투리와 매칭되지 않는 표준어도 딕셔너리에 추가

    eojeol_df = pd.DataFrame(eojeol_list, columns=None)
    eojeol_dict = eojeol_df.to_dict()
    eojeol_dict = eojeol_dict[0]
    eojeol_dict = {v:k for k,v in eojeol_dict.items()}

    #print(eojeol_dict)

    return eojeol_dict

In [14]:
eojeol_dict = make_eojeol(index_to_standard, dialect_to_index)

7581
7581


In [191]:
with open('eojeol_dict.pickle','wb') as fw:
    pickle.dump(eojeol_dict, fw)
with open('eojeol_dict.pickle', 'rb') as fr:
    eojeol_dict = pickle.load(fr)

In [15]:
# 동의어 사전 만드는 함수
def make_synonym_list(eojeol_dict):
    synonym_list=[]
    for i in eojeol_dict.keys():
        if i[-1]=="0":
            for j in eojeol_dict.keys():
                tmp=i[:-1]
                if tmp==j:
                    synonym_list.append(tmp)
                    #print(i,j)
        else:
            pass
    return synonym_list

In [17]:
synonym_list = make_synonym_list(eojeol_dict)

In [194]:
with open('synonym_list_test.pickle','wb') as fw:
    pickle.dump(synonym_list, fw)
with open('synonym_list.pickle_test', 'rb') as fr:
    synonym_list = pickle.load(fr)

In [20]:
# 단어와 연관된 단어의 품사를 API를 사용해서 가져오기
def get_mods_pos(std_str, dlt_str):
    #print("dd")
    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    #accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    accessKey = "bd6d2bff-4e44-4035-8591-947fda262051"
    analysisCode = "dparse"

    text = std_str
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }
          
    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))

    words = []
    poses = []
    
    #try : 
    if tmp["result"] == -1:

        print(tmp)

    std_list = std_str.split(". ")
    dlt_list = dlt_str.split(". ")
    key = True
    ket = False
    for t in range(len(std_list)-1):
        #tmp["return_object"]["sentence"][t]

        std = std_list[t].split()
        dlt = dlt_list[t].split()

        try : 
            for i in range(len(tmp["return_object"]["sentence"][t]["dependency"])):
                dict_with_mods = tmp["return_object"]["sentence"][t]["dependency"][i]["mod"]
                # print("-------")
                # print(tmp["return_object"]["sentence"][t]["dependency"])
                #print("현재단어",tmp["return_object"]["sentence"][t]["dependency"][i]["text"])
                
                for m in dict_with_mods:
                    #print("모드있음")
                    #print(i)
                    if tmp["return_object"]["sentence"][t]["dependency"][i]["text"] != ".":
                        if std[i] != dlt[i]:
                            #print("사투리임")
                            # print(words)
                            
                            if dlt[i]+"0" in eojeol_dict.keys():
                                #print("동의어 사투리라서 두개 다 넣음", dlt[i], std[i])                            
                        
                                words.append(eojeol_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["text"]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][i]["label"]])
                                key=False

                                words.append(eojeol_dict[dlt[i]+"0"])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["label"]])
                                key=False

                                words.append(eojeol_dict[std[i]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["label"]])
                                key=False
                            # else:
                            #     # print("사투리 사전에 없음")

                        else:
                            #print("사투리아님")
                            if std[i]+"0" in eojeol_dict.keys():
                                
                                words.append(eojeol_dict[std[i]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["label"]])
                                key=False

                                words.append(eojeol_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["text"]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][i]["label"]])
                                key=False
            
            #print("성공")

        except:
            #print("실패")
            #print(len(words), len(poses))
            if key == True and len(words)>0:
                words.pop()
                print("Pop")
    #print(len(words), len(poses))

    # except:
    #     print(tmp)

    #print(words, poses)
    return words, poses

In [22]:
# 단어별 인덱스와 연결된 단어의 품사 인덱스 

def eojeol_dependence(raw_data, synonym_list):
    words=[]
    poses=[]
    std_str=""
    dlt_str=""
    count = 0
    num=0
    test = 0

    for i in raw_data["utterance"]:
        test += 1
        if test%10000 == 0:
            print("test",test)
        if test >600000:
            break


        # 문장
        std = i["standard_form"]
        dlt = i["dialect_form"]
        
        filter = re.compile('[^가-힣 +]')

        # 불용어 제거 문장
        dlt = filter.sub('', dlt)
        std = filter.sub('', std)

        # 문장 이어붙임
        total = dlt.split() + std.split()
        #print(total)

        # 이어붙인 문장의 어절을 돌면서
        for eojeol in total:
            #print(eojeol)
            if eojeol in synonym_list: # 동의어 목록에 있는 어절을 발견하면
                #print("동의어!!!!!!!", eojeol)
                # get_mods_pos로 넘길 str에 추가
                # print(std)
                # print(dlt)
                dlt_str = dlt_str + dlt + ". "
                std_str = std_str + std + ". "
                count = count + 1
                break

        if count >= 200: #문장 총 개수 22500개, 동의어가 있는 문장이 200개가 넘으면 API 사용 -> API 사용횟수 : 약 100회    
            num +=1
            #if num == 3:
            #print(std_str)
            #print(dlt_str)
            
            word, pos = get_mods_pos(std_str, dlt_str)
            #print(word, pos)
            if len(word) == len(pos):
                words.extend(word)
                poses.extend(pos)
            print(test, len(words), len(poses))
            count = 0
            std_str = ""
            dlt_str = ""
    #print("final_num",num)
    print("pos", len(poses), "words", len(words))

    df = pd.DataFrame(data = {"pos":poses, "word":words })
        
    return df

In [23]:
train_df = eojeol_dependence(raw_data, synonym_list)

377 159 159
713 288 288
1044 393 393
1471 506 506
1930 604 604
2363 711 711
2936 777 777
3409 850 850
3913 924 924
4400 995 995
4887 1066 1066
5369 1142 1142
5856 1215 1215
6356 1290 1290
6840 1361 1361
7336 1437 1437
7809 1510 1510
8313 1584 1584
8800 1655 1655
9287 1726 1726
9769 1802 1802
test 10000
10256 1875 1875
10756 1950 1950
11240 2021 2021
11736 2097 2097
12209 2170 2170
12713 2244 2244
13200 2315 2315
13687 2386 2386
14169 2462 2462
14656 2535 2535
15156 2610 2610
15640 2681 2681
16136 2757 2757
16609 2830 2830
17113 2904 2904
17600 2975 2975
18087 3046 3046
18569 3122 3122
19056 3195 3195
19556 3270 3270
test 20000
20040 3341 3341
20536 3417 3417
21009 3490 3490
21513 3564 3564
22000 3635 3635
22453 3734 3734
22961 3873 3873
23385 3980 3980
23782 4115 4115
24269 4251 4251
24923 4343 4343
25321 4433 4433
25773 4538 4538
26214 4631 4631
26564 4742 4742
26950 4831 4831
27395 4958 4958
27911 5025 5025
28375 5124 5124
28765 5254 5254
29139 5360 5360
29566 5460 5460
29996 5593 55

In [153]:
train_df.to_csv("train_df_1.csv")

In [25]:
train_df = pd.read_csv("train_df.csv")
train_df = train_df[["pos", "word"]]

In [28]:
len(set(train_df["word"]))

3961

In [29]:
len(set(train_df["word"]))

3961

In [30]:
# 원핫인코딩을 하기 위해 
index_word={} # 
num=0
for i in train_df["word"]:
    if i not in index_word.keys():
        index_word[i] = num
        num += 1
#print(index_word)
word_index_col=[]
for i in train_df["word"]:
    word_index_col.append(index_word[i])


In [31]:
train_df["word_index_col"] = word_index_col


In [32]:
train_df

,pos,word,word_index_col
0,0,1594,0
1,3,1594,0
2,0,9192,1
3,54,8394,2
4,59,218,3
...,...,...,...
142604,0,8076,721
142605,56,6678,29
142606,54,636,90
142607,54,11486,65


In [205]:
train_df.to_csv("train_df_1_word_index_col_test.csv")
train_df = pd.read_csv("train_df_1_word_index_col_test.csv")
train_df = train_df[["pos", "word","word_index_col"]]

In [34]:
synonym_xtrain = np_utils.to_categorical(train_df["pos"])
synonym_ytrain = np_utils.to_categorical(train_df["word_index_col"])
len(synonym_ytrain[0])

3961

In [32]:
synonym_xtrain = np_utils.to_categorical(train_df["pos"])
synonym_ytrain = np_utils.to_categorical(train_df["word_index_col"])
len(synonym_ytrain[0])

1573

In [33]:
print(synonym_xtrain.shape)
print(synonym_ytrain.shape)

(6133, 60)
(6133, 1573)


In [35]:
print(synonym_xtrain.shape)
print(synonym_ytrain.shape)

(142609, 61)
(142609, 3961)


In [36]:
synonym_model = Sequential()
synonym_model.add(Dense(512, input_dim = 61, activation="relu"))
synonym_model.add(Dense(256,activation="relu"))
synonym_model.add(Dense(128,activation="relu"))
synonym_model.add(Dense(3961, activation="softmax"))

In [37]:
synonym_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
synonym_model.fit(synonym_xtrain, synonym_ytrain, batch_size=500, epochs=50)

Epoch 1/50


In [537]:
synonym_model.save("synonym_model.h5")

In [34]:
from tensorflow.keras.models import load_model
synonym_model = load_model("synonym_model.h5")

In [34]:
synonym_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               31232     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 1573)              202917    
Total params: 398,373
Trainable params: 398,373
Non-trainable params: 0
_________________________________________________________________


In [88]:
# 단어와 연관된 단어의 품사 추출
def get_mods(sentence, num):
    #print(sentence, num)
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    
    #bd6d2bff-4e44-4035-8591-947fda262051
    analysisCode = "dparse"

    text = sentence
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))
    #print(tmp)
    dict_with_mods = tmp["return_object"]["sentence"][0]["dependency"]
    id_dict={}
    print(dict_with_mods)
    pos = ""
    id = ""
    if len(dict_with_mods[num]["mod"]) == 0:
        for i in range(len(dict_with_mods)):
            for j in dict_with_mods[i]["mod"]:
                # print(i)
                # print(j)
                # print(dict_with_mods[i]["mod"])
                if j == float(num):
                    print(i, j, float(num))
                    print(dict_with_mods[int(i)]["label"])
                    id_dict[dict_with_mods[int(i)]["weight"]] = int(i)
                    #print(id_dict[i]["id"])

    else:
        for i in dict_with_mods[num]["mod"]:
            id_dict[dict_with_mods[int(i)]["weight"]] = int(i)
            #print(i)
    id = id_dict[max(id_dict)]
    
    pos = dict_with_mods[id]["label"]
 

    # for i in dict_with_mods[num]["mod"]:
    #     id_dict[dict_with_mods[int(i)]["weight"]] = int(i)
    #     #print(i)
    # id = id_dict[max(id_dict)]
    
    # pos = dict_with_mods[id]["label"]

    print("dd",id_dict)
    return pos, dict_with_mods[id]["text"]

In [1]:
# 동의어 처리를 포함한 사투리 번역 함수
def predict_standard(sentence):
    global index_to_standard
    res = ""
    mark = ""
    if sentence[-1] in [".", "?", "!"]:
        mark = sentence[-1]
        sentence = sentence[:-1]

    sentence_sep = sentence.split()
    
        
    #print(sentence_sep)
    for i in range(len(sentence_sep)):
        #print(sentence_sep[i])
        if exist_in_dialect(sentence_sep[i]) == True: #사투리 사전에 있으면
            if sentence_sep[i] in eojeol_dict.keys(): #해당 단어가 전체 사전에 있으면
                
                #print(sentence_sep[i] )
                pos, _ = get_mods(sentence, i) #연관된 단어의 품사 받아옴
                pos_idx = pos_dict[pos] #품사의 인덱스
                
                word = sentence_sep[i]
                print(i, word) 
                print("0 포함", eojeol_dict[word+"0"])
                print("0 미포함", eojeol_dict[word])
                word0_idx = index_word[ eojeol_dict[word+"0"]] #단어0의 인덱스
                word_idx = index_word[eojeol_dict[word]] #단어의 인덱스
                #print(word0_idx)
                # temp = result[word_idx]
                # print(type(temp))

                one_hot = np_utils.to_categorical(pos_idx,61) #인덱스 배열의크기 지정 필요 size_of_dialect
                #print((one_hot))
                
                percentage = synonym_model.predict(np.array([one_hot])) # 동의어 처리 모델
                #print("ㅇㅇ",percentage[0])

                
                res_idx = max(percentage[0][word0_idx], percentage[0][word_idx]) # 둘 중 더 큰 확률값
                print("사투리확률", percentage[0][word0_idx])
                print("표준어확률", percentage[0][word_idx])
                
                if percentage[0][word0_idx] > percentage[0][word_idx]:

                    word = translate(word)
                else: 
                    pass

                res = res + " " + word
                #res = res + " " + index_to_standard[pred_index]
            else: #해당 단어가 전체 사전에 없으면 -> 동의어가 없으므로 바로 번역 모델에 넣음
                
                word = sentence_sep[i]
                res = res + " " + translate(word)
        else:
            word = sentence_sep[i]
            #print(word,"사투리가아님")
            res = res + " " + word
    if mark != "":
        res = res + mark
            
    return res

In [ ]:
predict_standard("내가 얼른 글로 갈게")

In [152]:
predict_standard("맞제 이거 약간 모의 투자 느낌.")

43


' 맞지 이거 약간 모의 투자 느낌.'

In [139]:
predict_standard("최고의 리더는 글을 쓴다.") #표준어 쓴다

[{'id': 0.0, 'text': '최고의', 'head': 1.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.58665}, {'id': 1.0, 'text': '리더는', 'head': 3.0, 'label': 'NP_SBJ', 'mod': [0.0], 'weight': 0.747763}, {'id': 2.0, 'text': '글을', 'head': 3.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.732951}, {'id': 3.0, 'text': '쓴다', 'head': -1.0, 'label': 'VP', 'mod': [1.0, 2.0], 'weight': 0.23426}]
3 쓴다
89
88112
사투리확률 2.4423378e-05
표준어확률 0.0022023644


' 최고의 리더는 글을 쓴다.'

In [138]:
predict_standard("최고의 리더니까 최고의 글을 쓴다 아이가?") #사투리 쓴다, 사투리 아이가

[{'id': 0.0, 'text': '최고의', 'head': 1.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.608439}, {'id': 1.0, 'text': '리더니까', 'head': 4.0, 'label': 'VNP', 'mod': [0.0], 'weight': 0.37775}, {'id': 2.0, 'text': '최고의', 'head': 3.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.56849}, {'id': 3.0, 'text': '글을', 'head': 4.0, 'label': 'NP_OBJ', 'mod': [2.0], 'weight': 0.688654}, {'id': 4.0, 'text': '쓴다', 'head': 5.0, 'label': 'VP', 'mod': [1.0, 3.0], 'weight': 0.336526}, {'id': 5.0, 'text': '아이가', 'head': -1.0, 'label': 'NP_SBJ', 'mod': [4.0], 'weight': 0.0141848}]
4 쓴다
89
88112
사투리확률 0.0025564772
표준어확률 4.2879005e-06
44
[{'id': 0.0, 'text': '최고의', 'head': 1.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.608439}, {'id': 1.0, 'text': '리더니까', 'head': 4.0, 'label': 'VNP', 'mod': [0.0], 'weight': 0.37775}, {'id': 2.0, 'text': '최고의', 'head': 3.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.56849}, {'id': 3.0, 'text': '글을', 'head': 4.0, 'label': 'NP_OBJ', 'mod': [2.0], 'weight': 0.688654}, {'id': 4.0, 'text': '쓴

' 최고의 리더니까 최고의 글을 쓰지 않아?'

In [153]:
predict_standard("게임을 그렇게 대충 해가지고 이기겠나?")

[{'id': 0.0, 'text': '게임을', 'head': 3.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.736509}, {'id': 1.0, 'text': '그렇게', 'head': 3.0, 'label': 'VP_AJT', 'mod': [], 'weight': 0.575393}, {'id': 2.0, 'text': '대충', 'head': 3.0, 'label': 'AP', 'mod': [], 'weight': 0.579023}, {'id': 3.0, 'text': '해가지고', 'head': 4.0, 'label': 'VP', 'mod': [0.0, 1.0, 2.0], 'weight': 0.532814}, {'id': 4.0, 'text': '이기겠나', 'head': -1.0, 'label': 'VP', 'mod': [3.0], 'weight': 0.10447}]
3 해가지고
643
156118
사투리확률 0.003530945
표준어확률 0.1711169
95


' 게임을 그렇게 대충 해가지고 이길까?'

In [154]:
predict_standard("게임을 그렇게 대충 해가지고 이기긌나?")

[{'id': 0.0, 'text': '게임을', 'head': 3.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.736048}, {'id': 1.0, 'text': '그렇게', 'head': 3.0, 'label': 'VP_AJT', 'mod': [], 'weight': 0.580144}, {'id': 2.0, 'text': '대충', 'head': 3.0, 'label': 'AP', 'mod': [], 'weight': 0.578524}, {'id': 3.0, 'text': '해가지고', 'head': 4.0, 'label': 'VP', 'mod': [0.0, 1.0, 2.0], 'weight': 0.516002}, {'id': 4.0, 'text': '이기긌나', 'head': -1.0, 'label': 'VP', 'mod': [3.0], 'weight': 0.100202}]
3 해가지고
643
156118
사투리확률 0.003530945
표준어확률 0.1711169


' 게임을 그렇게 대충 해가지고 이기긌나?'

In [157]:
predict_standard("어릴때 옆집 살던 가는 뭐하노?")

[{'id': 0.0, 'text': '어릴때', 'head': 4.0, 'label': 'VP', 'mod': [], 'weight': 0.238896}, {'id': 1.0, 'text': '옆집', 'head': 2.0, 'label': 'NP_SBJ', 'mod': [], 'weight': 0.0295369}, {'id': 2.0, 'text': '살던', 'head': 3.0, 'label': 'VP_MOD', 'mod': [1.0], 'weight': 0.42003}, {'id': 3.0, 'text': '가는', 'head': 4.0, 'label': 'VP_MOD', 'mod': [2.0], 'weight': 0.572229}, {'id': 4.0, 'text': '뭐하노', 'head': -1.0, 'label': 'VNP', 'mod': [0.0, 3.0], 'weight': 0.00104964}]
3 가는
281
155743
사투리확률 0.002120091
표준어확률 0.004655487


' 어릴때 옆집 살던 가는 뭐하노?'

In [141]:
predict_standard("너처럼 어린 아이가 무슨 운전 면허증을 따니?")

[{'id': 0.0, 'text': '너처럼', 'head': 6.0, 'label': 'NP_AJT', 'mod': [], 'weight': 0.711283}, {'id': 1.0, 'text': '어린', 'head': 2.0, 'label': 'VP_MOD', 'mod': [], 'weight': 0.607093}, {'id': 2.0, 'text': '아이가', 'head': 6.0, 'label': 'NP_SBJ', 'mod': [1.0], 'weight': 0.841032}, {'id': 3.0, 'text': '무슨', 'head': 5.0, 'label': 'DP', 'mod': [], 'weight': 0.38138}, {'id': 4.0, 'text': '운전', 'head': 5.0, 'label': 'NP', 'mod': [], 'weight': 0.327299}, {'id': 5.0, 'text': '면허증을', 'head': 6.0, 'label': 'NP_OBJ', 'mod': [3.0, 4.0], 'weight': 0.687231}, {'id': 6.0, 'text': '따니', 'head': -1.0, 'label': 'VP', 'mod': [0.0, 2.0, 5.0], 'weight': 0.0258534}]
2 아이가
37
148241
사투리확률 8.952507e-09
표준어확률 0.0059404573


' 너처럼 어린 아이가 무슨 운전 면허증을 따니?'